In [15]:
from pathlib import Path
import pandas as pd
import json
import numpy as np
# import shap

In [16]:
PATH = Path(r'C:\Users\niti.mishra\Documents\2_TDMDAL\projects\covid_tests\covid_tests')
# PATH = Path(r'/Users/peaceforlives/Documents/Projects/covid_symptoms')
DATA = Path.joinpath(PATH, 'data', 'old')
RESULTS = Path.joinpath(PATH, 'results')

## DATA ##

In [48]:
covid_data = pd.read_csv(Path.joinpath(DATA, 'RawData_IMOH_Sep_20.csv')) 

In [25]:
y = covid_data['testresult']
cols = ['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache', 'sixtiesplus', 'Gender', 'contact', 'abroad', 'Validation']
covid_data = covid_data[cols]

In [26]:
covid_data.index = covid_data['Validation']
covid_data.drop('Validation', inplace=True, axis=1)
covid_data.head()

,cough,fever,sore_throat,shortness_of_breath,head_ache,sixtiesplus,Gender,contact,abroad
Validation,,,,,,,,,
Training,0,1,0,0,0,1,1,0,0
Validation,0,1,0,0,0,0,0,0,0
Test,1,0,0,0,1,0,1,0,0
Test,1,0,0,0,0,0,0,1,0
Training,0,0,0,0,1,0,1,0,0


In [11]:
test = list(covid_data.sum(axis=1))

In [17]:
from collections import Counter

Counter(y)

Counter({1: 47238, 0: 35681})

In [21]:
(y.value_counts()/len(y))*100

1    56.968849
0    43.031151
Name: testresult, dtype: float64

In [22]:
test = {'cough': '0', 'fever': '1'}

# [[key for key in test[0].keys()], *[list(idx.values()) for idx in test]]

In [23]:
print("{" + "\n".join("{!r}: {!r},".format(k, v) for k, v in test.items()) + "}")

{'cough': '0',
'fever': '1',}


In [27]:
train = covid_data.index.get_loc('Training')
valid = covid_data.index.get_loc('Validation')
test = covid_data.index.get_loc('Test')

In [28]:
X_train = covid_data[train].values.tolist()
y_train = y[train].values.tolist()
print(len(y_train))

X_valid = covid_data[valid].values.tolist()
y_valid = y[valid].values.tolist()
print(len(y_valid))

X_test = covid_data[test].values.tolist()
y_test = y[test].values.tolist()
print(len(y_test))

41464
20744
20711


In [42]:
import numpy as np
import time
from sklearn.pipeline import Pipeline
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_auc_score


In [38]:
parameters = [
#     {'clf__C': ( np.logspace(-5, 1, 5) ),
#         'clf__penalty': ['l1', 'l2', 'none'] # regularization paramter
#         },
    {'clf__max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
        'clf__min_samples_split': [2, 5, 10],
        'clf__min_samples_leaf': [1, 2, 4]
        },
]

In [32]:
rf = RandomForestClassifier(max_depth=10, min_samples_leaf=4,
                                        min_samples_split=10,
                                        random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
roc_auc_score(y_test, y_pred)

0.7787989969376632

In [44]:
model = rf
params = parameters[0]
pipe = Pipeline([
            ('clf', model),
            ])
test_fold = [-1 for i in range(len(X_train))] + [0 for i in range(len(X_valid))]
ps = PredefinedSplit(test_fold=test_fold)
grid_search = GridSearchCV(estimator=pipe, param_grid=params, cv=ps, scoring='roc_auc')
# grid_search = GridSearchCV(estimator=pipe, param_grid=params, cv=k, scoring='roc_auc')

In [45]:
best_model = grid_search.fit(X_train+X_valid, np.concatenate( (y_train,y_valid)))
best_param = best_model.best_params_
best_score = best_model.best_score_
best_estimator = best_model.best_estimator_['clf']
print(best_param)
print(best_score)
print(best_estimator)

{'clf__max_depth': 10, 'clf__min_samples_leaf': 4, 'clf__min_samples_split': 2}
0.8345485153912919
RandomForestClassifier(max_depth=10, min_samples_leaf=4, random_state=42)


In [33]:
y_pred = rf.predict(X_train)
roc_auc_score(y_train, y_pred)

0.7769962684111129

In [ ]:
y_pred = rf.predict(X_test)
roc_auc_score(y_test, y_pred)

In [119]:
lr = LogisticRegression(random_state=42, penalty='none')
lr.fit(X_train, y_train)

LogisticRegression(penalty='none', random_state=42)

In [121]:
y_pred = lr.predict(X_test)
roc_auc_score(y_test, y_pred)

0.7743491881068725

In [122]:
lr.coef_

array([[-0.78111456,  0.46842223,  0.87043669,  1.03072509,  1.27444187,
         0.3634671 , -0.10938874,  0.61187191, -2.49559097]])

In [20]:
# # rename columns
# cols = ['test_date']
# cols.extend(covid_data.columns[1:] )
# covid_data.columns = cols

# # select columns
# cols = ['test_date', 'cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache',
#         'testresult', 'sixtiesplus', 'Gender', 'contact', 'abroad', 'Validation']
# covid_data = covid_data[cols]

# covid_data['test_date'] = pd.to_datetime(covid_data['test_date'], format='%Y-%m-%d')

# covid_data.head()

## RESULTS ##

In [11]:
RESULTS = Path.joinpath(PATH, 'results', 'no_missingdata',)
str(RESULTS)+'\\results.json'

'C:\\Users\\niti.mishra\\Documents\\2_TDMDAL\\projects\\covid_tests\\covid_tests\\results\\no_missingdata\\results.json'

In [17]:
res = pd.read_json(str(RESULTS)+'\\results.json', orient='records', lines=True)

In [18]:
res.set_index('time', inplace=True)

In [19]:
res.to_clipboard()

## SHAPLEY VALUE ##

In [33]:
import shap

ModuleNotFoundError: No module named 'shap'